In [1]:
import dask.array as da
import datashader as ds
import noise
import numpy as np
import pandas as pd
import xarray as xr

from mapshader.colors import colors
from mapshader.sources import MapSource, TileService, elevation_source
from mapshader.utils import build_previewer

In [2]:
def make_terrain(
    shape=(2**14, 2**14),
    scale=100.0,
    octaves=6,
    persistence=0.5,
    lacunarity=2.0,
    chunks=(8192, 8192)
):
    """
    Generate a pseudo-random terrain data dask array.
    Parameters
    ----------
    shape : int or tuple of int, default=(2**14, 2**14)
        Output array shape.
    scale : float, default=100.0
        Noise factor scale.
    octaves : int, default=6
        Number of waves when generating the noise.
    persistence : float, default=0.5
        Amplitude of each successive octave relative.
    lacunarity : float, default=2.0
        Frequency of each successive octave relative.
    chunks : int or tuple of int, default=(8192, 8192)
        Number of samples on each block.
    Returns
    -------
    terrain : xarray.DataArray
        2D array of generated terrain values.
    """
    def _func(arr, block_id=None):
        block_ystart = block_id[0] * arr.shape[0]
        block_xstart = block_id[1] * arr.shape[1]
        out = np.zeros(arr.shape)
        for i in range(out.shape[0]):
            for j in range(out.shape[1]):
                out[i][j] = noise.pnoise2(
                    (block_ystart + i)/scale,
                    (block_xstart + j)/scale,
                    octaves=octaves,
                    persistence=persistence,
                    lacunarity=lacunarity,
                    repeatx=1024,
                    repeaty=1024,
                    base=42,
                )
        return out
    data = (
        da.zeros(shape=shape, chunks=chunks, dtype=np.float32)
        .map_blocks(_func, dtype=np.float32)
    )
    cvs = ds.Canvas(
        x_range=(-20e6, 20e6),
        y_range=(-20e6, 20e6),
        plot_width=500,
        plot_height=500,
    )
    hack_agg = cvs.points(pd.DataFrame({'x': [], 'y': []}), 'x', 'y')
    agg = xr.DataArray(
        data,
        name='terrain',
        coords=hack_agg.coords,
        dims=hack_agg.dims,
        attrs={'res': 1},
    )
    return agg

In [3]:
data = make_terrain(shape=(500, 500))

In [4]:
type(data.data)

dask.array.core.Array

In [5]:
raster_src = MapSource.from_obj(elevation_source())

In [6]:
raster_src.data = data
raster_src.transforms = []
raster_src.cmap = colors['viridis']
raster_src.span = (-0.349882, 0.43575)
raster_src.load()

# ----------------------
# APPLYING TRANSFORMS Elevation
# ----------------------


In [7]:
service = TileService(source=raster_src)

In [8]:
from mapshader.flask_app import *

start_flask_app_jupyter([service, ])

 * Serving Flask app 'mapshader.flask_app' (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
127.0.0.1 - - [31/May/2021 01:06:09] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [31/May/2021 01:06:09] "GET /favicon.ico HTTP/1.1" 404 -
127.0.0.1 - - [31/May/2021 01:06:11] "GET /elevation-tile HTTP/1.1" 200 -
127.0.0.1 - - [31/May/2021 01:06:15] "GET /psutil HTTP/1.1" 200 -
127.0.0.1 - - [31/May/2021 01:06:16] "GET /elevation-tile/tile/2/0/1 HTTP/1.1" 200 -
127.0.0.1 - - [31/May/2021 01:06:16] "GET /elevation-tile/tile/2/1/1 HTTP/1.1" 200 -
127.0.0.1 - - [31/May/2021 01:06:16] "GET /elevation-tile/tile/1/1/0 HTTP/1.1" 200 -
127.0.0.1 - - [31/May/2021 01:06:16] "GET /elevation-tile/tile/1/0/0 HTTP/1.1" 200 -
127.0.0.1 - - [31/May/2021 01:06:16] "GET /elevation-tile/tile/1/0/1 HTTP/1.1" 200 -
127.0.0.1 - - [31/May/2021 01:06:16] "GET /elevation-tile/tile/1/1/1 HTTP/1.1" 200 -
127.0.0.1 - - [31/May/2021 01:06:17] "GET /psutil HTTP/1.1" 200 -
127.0.0.1 - - [31/May/2021 01:06:18] "GET /elevation-tile/tile/2/3/1 HTTP/1.1" 200 -
127.0.0.

In [9]:
from bokeh.plotting import show
from bokeh.io import output_notebook


output_notebook()


show(build_previewer(service, host='http://127.0.0.1:5000'))

Loading BokehJS ...

127.0.0.1 - - [31/May/2021 01:07:57] "GET /elevation-tile/tile/1/0/0 HTTP/1.1" 200 -
127.0.0.1 - - [31/May/2021 01:07:57] "GET /elevation-tile/tile/1/1/0 HTTP/1.1" 200 -
127.0.0.1 - - [31/May/2021 01:07:57] "GET /elevation-tile/tile/2/3/1 HTTP/1.1" 200 -
127.0.0.1 - - [31/May/2021 01:07:57] "GET /elevation-tile/tile/2/2/1 HTTP/1.1" 200 -
127.0.0.1 - - [31/May/2021 01:07:57] "GET /elevation-tile/tile/1/0/1 HTTP/1.1" 200 -
127.0.0.1 - - [31/May/2021 01:07:57] "GET /elevation-tile/tile/1/1/1 HTTP/1.1" 200 -
127.0.0.1 - - [31/May/2021 01:08:00] "GET /elevation-tile/tile/2/2/0 HTTP/1.1" 200 -
127.0.0.1 - - [31/May/2021 01:08:00] "GET /elevation-tile/tile/2/3/0 HTTP/1.1" 200 -
127.0.0.1 - - [31/May/2021 01:08:00] "GET /elevation-tile/tile/2/3/2 HTTP/1.1" 200 -
127.0.0.1 - - [31/May/2021 01:08:00] "GET /elevation-tile/tile/2/3/3 HTTP/1.1" 200 -
127.0.0.1 - - [31/May/2021 01:08:00] "GET /elevation-tile/tile/2/2/3 HTTP/1.1" 200 -
127.0.0.1 - - [31/May/2021 01:08:00] "GET /elevation-tile/tile/2/